<a href="https://colab.research.google.com/github/mojoblue/Kaggle_BBC_news_classification/blob/master/BBC_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from gensim.parsing.preprocessing import preprocess_string
import pandas as pd
import io

Mount drive files

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Do preprocessing news data text using gensim
https://radimrehurek.com/gensim/parsing/preprocessing.html

In [0]:
category2Index = {
    "sport" : 0,
    "tech" : 1,
    "business" : 2,
    "entertainment" : 3,
    "politics" : 4
}

In [0]:
index2Category = {
    0 : "sport",
    1 : "tech",
    2 : "business",
    3 : "entertainment",
    4 : "politics"
}

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/learn-ai-bbc/BBC News Train.csv', encoding="utf-8")

In [0]:
# preprocessed news data( Text => preprocessed Word ) and category2index
df['Text'] = df['Text'].map(lambda x : preprocess_string(x))
df['Category'] = df['Category'].map(lambda x: category2Index[x])

In [21]:
print(df['Text'][:2])

0    [worldcom, boss, launch, defenc, lawyer, defen...
1    [german, busi, confid, slide, german, busi, co...
Name: Text, dtype: object


In [22]:
print(df['Category'][:2])

0    2
1    2
Name: Category, dtype: int64


Load GloVe model twitter 100 dimension


In [0]:
def loadGloveModel(gloveFile):
  print("Loading GloVe Model")
  model = {}
  with open(gloveFile) as f:
    for line in f:
      splitLine = line.split()
      word = splitLine[0]
      embedding = np.array([float(val) for val in splitLine[1:]])
      model[word] = embedding
  print("Done.", len(model), " words loaded!")
  return model

In [24]:
gloveModel = loadGloveModel('/content/gdrive/My Drive/learn-ai-bbc/glove.twitter.27B.100d.txt')

Loading GloVe Model
Done. 1193515  words loaded!


Make baseline model using BOW

In [0]:
def makeBOW(dataframe):
  BOW_dict=dict()
  BOW_word_dict = dict()
  for wordList in df['Text']:
    for word in wordList:
      BOW_dict.setdefault(word, 0)
      BOW_dict[word] += 1
  for i, item in enumerate(BOW_dict.keys()):
    BOW_word_dict[item] = i
    
  BOW_all_sentences_list=[]
  for wordList in df['Text']:
    BOW_sentence_list = [0 for i in range(len(BOW_dict.keys()))]
    for word in wordList:
      BOW_sentence_list[BOW_word_dict[word]] += 1
    BOW_all_sentences_list.append(BOW_sentence_list)
  return BOW_all_sentences_list

In [0]:
def oneHotEncoding(dataframe, label_length):
  y_one_hot_data = []
  for label in dataframe:
    one_hot_data = [0 for i in range(label_length)]
    one_hot_data[label] = 1
    y_one_hot_data.append(one_hot_data)
  return np.array(y_one_hot_data)

In [0]:
BOW_list = makeBOW(df)

In [63]:
print(BOW_list[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 5, 1, 2, 3, 2, 2, 1, 1, 1, 1, 5, 1, 1, 1, 1, 5, 3, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [53]:
len(BOW_list[0])

15781

In [54]:
len(df['Category'])

1490

In [0]:
y_label = oneHotEncoding(df['Category'], 5)

In [94]:
y_label[:3]

array([[0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0]])

In [96]:
y_label.shape

(1490, 5)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

In [0]:
model = Sequential()

In [106]:
model.add(Dense(100, input_dim=len(BOW_list[0]), activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [108]:
np.array(BOW_list).shape

(1490, 15781)

In [109]:
train = model.fit(np.array(BOW_list), y_label, epochs=100, batch_size=32)

Epoch 1/100
1490/1490 [==============================] - 2s 1ms/step - loss: 0.4978 - acc: 0.8785
Epoch 2/100
1490/1490 [==============================] - 1s 849us/step - loss: 0.0443 - acc: 0.9913
Epoch 3/100
1490/1490 [==============================] - 1s 856us/step - loss: 0.0176 - acc: 0.9980
Epoch 4/100
1490/1490 [==============================] - 1s 845us/step - loss: 0.0036 - acc: 0.9987
Epoch 5/100
1490/1490 [==============================] - 1s 856us/step - loss: 4.6074e-04 - acc: 1.0000
Epoch 6/100
1490/1490 [==============================] - 1s 845us/step - loss: 1.8744e-04 - acc: 1.0000
Epoch 7/100
1490/1490 [==============================] - 1s 818us/step - loss: 1.8542e-04 - acc: 1.0000
Epoch 8/100
1490/1490 [==============================] - 1s 796us/step - loss: 1.2410e-05 - acc: 1.0000
Epoch 9/100
1490/1490 [==============================] - 1s 807us/step - loss: 5.4021e-06 - acc: 1.0000
Epoch 10/100
1490/1490 [==============================] - 1s 824us/step - loss: 1.